In [1]:
import pandas as pd
import re


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/anaconda3/lib/python3.12/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/lib/python3.12/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/sit

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/anaconda3/lib/python3.12/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/lib/python3.12/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/sit

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



In [2]:
df = pd.read_csv('data/raw/Scats Data October 2006.csv', skiprows=1)

In [3]:
print(df.columns.tolist())


['SCATS Number', 'Location', 'CD_MELWAY', 'NB_LATITUDE', 'NB_LONGITUDE', 'HF VicRoads Internal', 'VR Internal Stat', 'VR Internal Loc', 'NB_TYPE_SURVEY', 'Date', 'V00', 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'Unnamed: 106', 'Unnamed: 107', 'Unnamed: 108']


In [4]:
df.head() 

,SCATS Number,Location,CD_MELWAY,NB_LATITUDE,NB_LONGITUDE,HF VicRoads Internal,VR Internal Stat,VR Internal Loc,NB_TYPE_SURVEY,Date,...,V89,V90,V91,V92,V93,V94,V95,Unnamed: 106,Unnamed: 107,Unnamed: 108
0,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,1/10/2006,...,66,81,50,59,47,29,34,NaN,NaN,NaN
1,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,2/10/2006,...,114,80,60,62,48,44,26,NaN,NaN,NaN
2,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,3/10/2006,...,86,93,90,73,57,29,40,NaN,NaN,NaN
3,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,4/10/2006,...,101,113,90,78,66,52,44,NaN,NaN,NaN
4,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,5/10/2006,...,113,99,91,61,55,49,36,NaN,NaN,NaN


In [5]:
df_cleaned = df.drop(columns=[col for col in df.columns if "Unnamed" in col])

In [6]:
id_cols = ['SCATS Number', 'Location', 'Date']
volume_cols = [col for col in df_cleaned.columns if re.fullmatch(r'V\d+', col)]

In [7]:
df_long = df_cleaned.melt(id_vars=id_cols, value_vars=volume_cols,
                          var_name='Interval', value_name='Volume')

In [8]:
def interval_to_time(interval):
    index = int(interval[1:])   
    minutes = index * 15
    hour = minutes // 60
    minute = minutes % 60
    return f"{hour:02d}:{minute:02d}"

df_long['Time'] = df_long['Interval'].apply(interval_to_time)

In [9]:
df_long.head()

,SCATS Number,Location,Date,Interval,Volume,Time
0,970,WARRIGAL_RD N of HIGH STREET_RD,1/10/2006,V00,86,00:00
1,970,WARRIGAL_RD N of HIGH STREET_RD,2/10/2006,V00,32,00:00
2,970,WARRIGAL_RD N of HIGH STREET_RD,3/10/2006,V00,26,00:00
3,970,WARRIGAL_RD N of HIGH STREET_RD,4/10/2006,V00,32,00:00
4,970,WARRIGAL_RD N of HIGH STREET_RD,5/10/2006,V00,40,00:00


In [10]:
# Bước 6: Tạo datetime chuẩn
df_long['Datetime'] = pd.to_datetime(df_long['Date'] + ' ' + df_long['Time'], dayfirst=True)

# Bước 7: Đổi tên cột cho dễ xử lý
df_long = df_long.rename(columns={'SCATS Number': 'SiteID'})

# Bước 8: Loại bỏ dòng có Volume bị thiếu
df_long = df_long.dropna(subset=['Volume'])

# Bước 9: Sắp xếp và reset lại index
df_long = df_long.sort_values(by=['SiteID', 'Datetime']).reset_index(drop=True)

In [11]:
df_long.head()

,SiteID,Location,Date,Interval,Volume,Time,Datetime
0,970,WARRIGAL_RD N of HIGH STREET_RD,1/10/2006,V00,86,00:00,2006-10-01 00:00:00
1,970,HIGH STREET_RD E of WARRIGAL_RD,1/10/2006,V00,37,00:00,2006-10-01 00:00:00
2,970,WARRIGAL_RD S of HIGH STREET_RD,1/10/2006,V00,92,00:00,2006-10-01 00:00:00
3,970,HIGH STREET_RD W of WARRIGAL_RD,1/10/2006,V00,47,00:00,2006-10-01 00:00:00
4,970,WARRIGAL_RD N of HIGH STREET_RD,1/10/2006,V01,83,00:15,2006-10-01 00:15:00


In [12]:
df_long.to_csv("data/processed/cleaned_scats_data.csv", index=False)